# 1

In [ ]:

import openai
import re
import pandas as pd
from sqlalchemy import create_engine, text


# 设置 API 密钥和基本 URL
client = openai.OpenAI(
    api_key="sk-",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

# 初始化对话历史
conversation_history = [
    {"role": "system", "content": """
        你是一个SQL查询生成助手。数据库的表名是emails2，列名和数据类型如下：
        - Message-IDd (VARCHAR(255)): 邮件唯一标识符，例如 '<18782981.1075855378110.JavaMail.evans@thyme'。
        - Fromd (VARCHAR(255)): 发件人邮件地址，例如 'phillip.allen@enron.co'。
        - Tod (VARCHAR(255)): 收件人邮件地址，例如 'tim.belden@enron.co'。
        - Subjectd (VARCHAR(255)): 邮件主题，可以为空，例如 'Re: test'。
        - contentd (LONGTEXT): 邮件内容，可能包含文本、换行符等。
        - Timed (DATETIME): 邮件日期和时间，格式例如 "2001-05-14 16:39:00"。

        请确保生成的SQL语句以

sql开头并以

结尾。
    """}
]
# 为了避免混淆，列名全加了d
# ALTER TABLE emails2
# CHANGE COLUMN Message-ID Message-IDd VARCHAR(255),
# CHANGE COLUMN From Fromd VARCHAR(255),
# CHANGE COLUMN To Tod VARCHAR(255),
# CHANGE COLUMN Subject Subjectd VARCHAR(255),
# CHANGE COLUMN content contentd LONGTEXT,
# CHANGE COLUMN Date Dated VARCHAR(50);

# 数据库配置
database_username = ''
database_password = ''
database_ip = '127.0.0.1'
database_name = 'myemail'
database_connection = f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}'
# 创建数据库引擎
engine = create_engine(database_connection)


def chat_with_model(user_input):
    # 将用户输入添加到对话历史
    conversation_history.append({"role": "user", "content": user_input})
    # 调用大模型进行对话
    response = client.chat.completions.create(
        messages=conversation_history,
        model=""
    )
    # 获取模型回复
    model_reply = response.choices[0].message.content.strip()
    # 将模型回复添加到对话历史
    conversation_history.append({"role": "assistant", "content": model_reply})
    return model_reply


def extract_sql_query(response):
    # 使用正则表达式提取SQL代码块
    match = re.search(r"sql(.*?)", response, re.DOTALL)
    if match:
        sql_query = match.group(1).strip()
        return sql_query
    else:
        raise ValueError("模型返回的SQL语句格式不正确")


def natural_language_to_sql(natural_language_query):
    user_input = f"按照自然语言的意思转换为SQL语句: {natural_language_query}"
    sql_query_response = chat_with_model(user_input)
    sql_query = extract_sql_query(sql_query_response)
    sql_query = f"USE myemail; {sql_query}"
    return sql_query


def execute_sql_query(sql_query):
    sql_query = sql_query.replace("\n", " ")
    sql_statements = sql_query.split(";")
    with engine.connect() as connection:
        for statement in sql_statements:
            if statement.strip():
                result = connection.execute(text(statement.strip()))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        return df


def summarize_email_contents(df):
    # 检查是否有 'contentd' 列
    if 'contentd' in df.columns:
        # 获取所有contentd的内容，合并成一个字符串，最多？个字符
        content = " ".join(df['contentd'].tolist())[:2047]

        # 调用大模型进行内容总结
        summary = chat_with_model_for_summary(content)
        return summary
    else:
        return "不需要总结。"


def chat_with_model_for_summary(content):
    # 构造请求内容
    user_input = f"请概括性地总结以下邮件内容：{content}"
    # 调用大模型进行总结
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model=""
    )
    # 获取模型回复
    summary = response.choices[0].message.content.strip()
    return summary


# 持续对话
print("开始对话")
while True:
    print("================================")
    user_input = input("你: ")
    if user_input.lower() in ["退出", "q", "quit", "exit"]:
        print("对话结束。")
        break
    try:
        sql_query = natural_language_to_sql(user_input)  # 将自然语言转换为SQL语句
        print(f"生成的SQL查询:\n{sql_query}")
        df = execute_sql_query(sql_query)  # 执行SQL查询并返回结果到一个DataFrame里，后续有无限操作可能性
        print("查询结果:【节选】")
        print(df.shape)
        print(df.head())

        # 如果 df 中有 'contentd' 列，则进行内容总结
        if 'contentd' in df.columns:
            summary = summarize_email_contents(df)
            print("邮件内容总结:")
            print(summary)
        else:
            print("DataFrame中没有'contentd'列，不需要总结。")

    except Exception as e:
        print(f"出错了: {e}")

# 2

In [ ]:
# 2
"""
我想看看phillip.allen@enron.co发给tim.belden@enron.co的全部邮件
给我2000年10月8日12点到22时的所有邮件内容
2000年phillip.allen@enron.co的邮箱有没有谈到有关Enron Energy的内容
"""

# 为了避免混淆，列名全加了d
# ALTER TABLE emails2
# CHANGE COLUMN `Message-ID` `Message-IDd` VARCHAR(255),
# CHANGE COLUMN `From` `Fromd` VARCHAR(255),
# CHANGE COLUMN `To` `Tod` VARCHAR(255),
# CHANGE COLUMN `Subject` `Subjectd` VARCHAR(255),
# CHANGE COLUMN `content` `contentd` LONGTEXT,
# CHANGE COLUMN `Date` `Dated` VARCHAR(50);


import openai
import re
import pandas as pd
from sqlalchemy import create_engine, text
import streamlit as st


# 设置 API 密钥和基本 URL
client = openai.OpenAI(
    api_key="sk-",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

# 初始化对话历史
conversation_history = [
    {"role": "system", "content": """
        你是一个SQL查询生成助手。数据库的表名是emails2，列名和数据类型如下：
        - `Message-IDd` (VARCHAR(255)): 邮件唯一标识符，例如 '<18782981.1075855378110.JavaMail.evans@thyme'。
        - `Fromd` (VARCHAR(255)): 发件人邮件地址，例如 'phillip.allen@enron.co'。
        - `Tod` (VARCHAR(255)): 收件人邮件地址，例如 'tim.belden@enron.co'。
        - `Subjectd` (VARCHAR(255)): 邮件主题，可以为空，例如 'Re: test'。
        - `contentd` (LONGTEXT): 邮件内容，可能包含文本、换行符等。
        - `Timed` (DATETIME): 邮件日期和时间，格式例如 "2001-05-14 16:39:00"。

        请确保生成的SQL语句以```sql开头并以```结尾。（必须是MySQL方言）
    """}
]

# 数据库配置
database_username = ''
database_password = ''
database_ip = '127.0.0.1'
database_name = 'myemail'
database_connection = f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}'
# 创建数据库引擎
engine = create_engine(database_connection)


def chat_with_model(user_input):
    # 将用户输入添加到对话历史
    conversation_history.append({"role": "user", "content": user_input})
    # 调用大模型进行对话
    response = client.chat.completions.create(
        messages=conversation_history,
        model=""
    )
    # 获取模型回复
    model_reply = response.choices[0].message.content.strip()
    # 将模型回复添加到对话历史
    conversation_history.append({"role": "assistant", "content": model_reply})
    return model_reply


def extract_sql_query(response):
    # 使用正则表达式提取SQL代码块
    match = re.search(r"```sql(.*?)```", response, re.DOTALL)
    if match:
        sql_query = match.group(1).strip()
        return sql_query
    else:
        raise ValueError("模型返回的SQL语句格式不正确")


def natural_language_to_sql(natural_language_query):
    user_input = f"按照自然语言的意思转换为SQL语句: {natural_language_query}"
    sql_query_response = chat_with_model(user_input)
    sql_query = extract_sql_query(sql_query_response)
    sql_query = f"USE myemail;\n{sql_query}"
    return sql_query


def execute_sql_query(sql_query):
    sql_query = sql_query.replace("\n", " ")
    sql_statements = sql_query.split(";")
    with engine.connect() as connection:
        for statement in sql_statements:
            if statement.strip():
                result = connection.execute(text(statement.strip()))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        return df


def chat_with_model_for_summary(content):
    # 构造请求内容
    user_input = f"请概括性地总结这些邮件内容：{content}"
    # 调用大模型进行总结
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model=""
    )
    # 获取模型回复
    summary = response.choices[0].message.content.strip()
    return summary


def summarize_email_contents(df):
    # 检查是否有 'contentd' 列
    if 'contentd' in df.columns:
        # 获取所有contentd的内容，合并成一个字符串，最多？个字符
        content = " ".join(df['contentd'].tolist())[:2047]
        # 调用大模型进行内容总结
        summary = chat_with_model_for_summary(content)
        return summary
    else:
        return "不需要总结。"


# Streamlit 应用
st.title("邮件分析任务")
# 添加功能选择
option = st.selectbox("选择功能:", ["此处下拉菜单选择功能", "大模型邮件总结", "特定主题的邮件查询", "特定发件人的邮件聚合"])

if option == "此处下拉菜单选择功能":
    pass

elif option == "大模型邮件总结":
    st.markdown("# 大模型邮件总结")
    st.markdown("请输入自然语言查询，并获取对应的 SQL 查询结果。")

    user_input = st.text_input("请在此输入:")

    if st.button("执行查询"):
        try:
            sql_query = natural_language_to_sql(user_input)  # 将自然语言转换为SQL语句
            st.markdown("## SQL 语句:")
            st.code(sql_query, language='sql')
            df = execute_sql_query(sql_query)  # 执行SQL查询并返回结果到一个DataFrame里，后续有无限操作可能性
            st.markdown("## 查询结果:")
            st.dataframe(df)

            # 如果 df 中有 'contentd' 列，则进行内容总结
            if 'contentd' in df.columns:
                summary = summarize_email_contents(df)
                st.markdown("## 邮件内容总结:")
                st.markdown(summary)
            else:
                st.write("这个不需要总结。")
        except Exception as e:
            st.write(f"* 出错了: {e} *")

elif option == "特定主题的邮件查询":
    st.markdown("# 特定主题的邮件查询")
    st.markdown("* 针对特定主题的邮件查询，如\"最近一个月xxx@xxx.com的邮箱有没有谈到xxx内容\"。")


elif option == "特定发件人的邮件聚合":
    st.markdown("# 特定发件人的邮件聚合")
    st.markdown(
        "* 针对特定发件人的邮件聚合，如\"罗列出最近一个月xxx@xxx.com邮箱与其他人讨论的主要事项的前因后果\"，将完整的邮件沟通链条提取并进行总结,其中涉及内容的前因后果。")


# 侧边栏
st.sidebar.markdown("""
    <style>
    @keyframes example {
        0%   {color: red;}
        25%  {color: orange;}
        50%  {color: blue;}
        100% {color: green;}
    }
    .animated-header {
        font-size: 2em;
        font-weight: bold;
        animation-name: example;
        animation-duration: 10s;
        animation-iteration-count: infinite;
    }
    </style>
    <div class="animated-header">项目说明</div>
    <p>作者：</p>
    """, unsafe_allow_html=True)
st.sidebar.markdown("![1](https://raw.githubusercontent.com/LukiRyan/TyporaImageBox/main/img/202407191058169.png)")


# 3

In [ ]:
"""
我想看看phillip.allen@enron.co发给tim.belden@enron.co的全部邮件
给我2000年10月8日12点到22时的所有邮件内容
2000年phillip.allen@enron.co的邮箱有没有谈到有关Enron Energy的内容
"""

# 为了避免混淆，列名全加了d
# ALTER TABLE emails2
# CHANGE COLUMN `Message-ID` `Message-IDd` VARCHAR(255),
# CHANGE COLUMN `From` `Fromd` VARCHAR(255),
# CHANGE COLUMN `To` `Tod` VARCHAR(255),
# CHANGE COLUMN `Subject` `Subjectd` VARCHAR(255),
# CHANGE COLUMN `content` `contentd` LONGTEXT,
# CHANGE COLUMN `Date` `Dated` VARCHAR(50);

import openai
import re
import pandas as pd
import streamlit as st
from sqlalchemy import create_engine, text

# 数据库配置
database_username = ''
database_password = ''
database_ip = '127.0.0.1'
database_name = 'myemail'
database_connection = f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}'
# 创建数据库引擎
engine = create_engine(database_connection)

# 设置 API 密钥和基本 URL
client = openai.OpenAI(
    api_key="sk-",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

# 初始化对话历史
conversation_history_1 = [
    {"role": "system", "content": """
        你是一个SQL查询生成助手。数据库的表名是emails2，列名和数据类型如下：
        - `Message-IDd` (VARCHAR(255)): 邮件唯一标识符，例如 '<18782981.1075855378110.JavaMail.evans@thyme'。
        - `Fromd` (VARCHAR(255)): 发件人邮件地址，例如 'phillip.allen@enron.co'。
        - `Tod` (VARCHAR(255)): 收件人邮件地址，例如 'tim.belden@enron.co'。
        - `Subjectd` (VARCHAR(255)): 邮件主题，可以为空，例如 'Re: test'。
        - `contentd` (LONGTEXT): 邮件内容，可能包含文本、换行符等。
        - `Timed` (DATETIME): 邮件日期和时间，格式例如 "2001-05-14 16:39:00"。

        请确保生成的SQL语句以```sql开头并以```结尾。（必须是MySQL方言）
    """}
]
conversation_history_2 = []


demand_1 = "请概括性地总结这些邮件内容："
demand_2 = ""


def chat_with_model(i_say, demand):
    """
    :param i_say: 每次对话的内容
    :param user_input: 初始化对话的内容
    :return: 模型的回复
    """
    # 将用户输入添加到对话历史
    user_input = demand + i_say
    # 调用大模型进行对话
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="qwen-turbo"
    )
    # 获取模型回复
    model_reply = response.choices[0].message.content.strip()
    # 将模型回复添加到对话历史
    user_input.append({"role": "assistant", "content": model_reply})
    return model_reply


def chat_with_model_plus(instruct, freestyle):
    """
    同chat_with_model，去掉初始化语句，加入每次对话前导自然语言。
    :param freestyle: 后导的自然语言
    :param instruct: 前导的自然语言，一般为需求
    :return:
    """
    # 构造请求内容
    user_input = instruct + f"{freestyle}"
    # 调用大模型进行总结
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="qwen-turbo"
    )
    # 获取模型回复
    model_reply = response.choices[0].message.content.strip()
    return model_reply


def extract_sql_query(complex_str):
    """
    从字符串提取出的SQL语句
    :param complex_str: 字符串
    :return: 字符串提取出的SQL语句
    """
    # 使用正则表达式提取SQL代码块
    match = re.search(r"```sql(.*?)```", complex_str, re.DOTALL)
    if match:
        sql_query = match.group(1).strip()
        return sql_query
    else:
        raise ValueError("模型返回的SQL语句格式不正确")


def natural_language_to_sql(natural_language, instruct):
    """
    相当于整合了：用户的自然语言-->大模型返回的sql语句
    :param natural_language: 用户输入的自然语言
    :param instruct: 专门传给chat_with_model的参数（初始化对话的内容）
    :return: SQL语句
    """
    user_input = f"按照自然语言的意思转换为SQL语句: {natural_language}"
    sql_query_response = chat_with_model(user_input, instruct)
    sql_query = extract_sql_query(complex_str=sql_query_response)
    sql_query = f"USE myemail;\n{sql_query}"
    return sql_query


def execute_sql_query(sql_query):
    """
    执行SQL语句：大模型返回的sql语句-->执行sql查询
    :param sql_query: SQL语句
    :return: 以df的格式返回sql查询结果
    """
    sql_query = sql_query.replace("\n", " ")
    sql_statements = sql_query.split(";")
    with engine.connect() as connection:
        for statement in sql_statements:
            if statement.strip():
                result = connection.execute(text(statement.strip()))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        return df


def df_to_model_reply(df, instruct):
    """
    在查询返回的df结果里，剥离出content并整合，交给大模型后得到回复
    :param df: 表格结果
    :param instruct: 专门传给chat_with_model_plus的参数（每次对话前导自然语言）
    :return: 大模型的回复
    """
    # 检查是否有 'contentd' 列
    if 'contentd' in df.columns:
        # 获取所有contentd的内容，合并成一个字符串，最多？个字符
        content = " ".join(df['contentd'].tolist())[:2047]
        # 调用大模型交流并获取回复
        model_reply = chat_with_model_plus(instruct=instruct, freestyle=content)
        return model_reply
    else:
        return "不需要总结。"


# Streamlit 应用
st.title("2024 邮件分析任务")
# 添加功能选择
option = st.selectbox("选择功能:", ["下拉选择功能", "大模型邮件总结", "特定主题的邮件查询", "特定发件人的邮件聚合"])

if option == "下拉选择功能":
    pass

elif option == "大模型邮件总结":
    st.markdown("# 大模型邮件总结")
    st.markdown(
        "* 利用Qwen等大语言模型，对检索到的邮件内容进行智能总结与生成,可以使用ollama结合qwen 0.5b的模型在本地运行。\n* 根据用户输入的检索条件(如关键词、时间范围、发件人等)，返回符合条件的邮件总结结果。\n* 总结结果要突出重点，忠实原文，语言通顺自然")
    user_input = st.text_input("请在此输入:")
    if st.button("执行查询"):
        try:
            sql_query = natural_language_to_sql(natural_language=user_input, instruct=conversation_history_1)  # 将自然语言转换为SQL语句
            st.markdown("## SQL 语句:")
            st.code(sql_query, language='sql')
            df = execute_sql_query(sql_query)  # 执行SQL查询并返回结果到一个DataFrame里，后续有无限操作可能性
            st.markdown("## 查询结果:")
            st.dataframe(df)
            # 如果 df 中有 'contentd' 列，则进行内容总结
            if 'contentd' in df.columns:
                summary = df_to_model_reply(df=df, instruct=demand_1)
                st.markdown("## 邮件内容总结:")
                st.markdown(summary)
            else:
                st.write("这个不需要总结。")
        except Exception as e:
            st.write(f"* 出错了: {e} *")

elif option == "特定主题的邮件查询":
    st.markdown("# 特定主题的邮件查询")
    st.markdown("* 针对特定主题的邮件查询，如\"最近一个月xxx@xxx.com的邮箱有没有谈到xxx内容\"。")


elif option == "特定发件人的邮件聚合":
    st.markdown("# 特定发件人的邮件聚合")
    st.markdown(
        "* 针对特定发件人的邮件聚合，如\"罗列出最近一个月xxx@xxx.com邮箱与其他人讨论的主要事项的前因后果\"，将完整的邮件沟通链条提取并进行总结,其中涉及内容的前因后果。")

# 侧边栏
st.sidebar.markdown("""
    <style>
    @keyframes example {
        0%   {color: red;}
        25%  {color: orange;}
        50%  {color: blue;}
        100% {color: green;}
    }
    .animated-header {
        font-size: 2em;
        font-weight: bold;
        animation-name: example;
        animation-duration: 10s;
        animation-iteration-count: infinite;
    }
    </style>
    <div class="animated-header">项目说明</div>
    <p>作者：</p>
    """, unsafe_allow_html=True)
st.sidebar.markdown("![1](https://raw.githubusercontent.com/LukiRyan/TyporaImageBox/main/img/202407191058169.png)")


# 4 good

In [ ]:
# 用户的自然语言-->大模型返回的sql语句-->执行sql查询-->再用大模型对查询内容总结
"""
我想看看phillip.allen@enron.co发给tim.belden@enron.co的全部邮件
给我2000年10月8日12点到22时的所有邮件内容
2000年phillip.allen@enron.co的邮箱有没有谈到有关Enron Energy的内容
"""

# 为了避免混淆，列名全加了d
# ALTER TABLE emails2
# CHANGE COLUMN `Message-ID` `Message-IDd` VARCHAR(255),
# CHANGE COLUMN `From` `Fromd` VARCHAR(255),
# CHANGE COLUMN `To` `Tod` VARCHAR(255),
# CHANGE COLUMN `Subject` `Subjectd` VARCHAR(255),
# CHANGE COLUMN `content` `contentd` LONGTEXT,
# CHANGE COLUMN `Date` `Dated` VARCHAR(50);


import openai
import re
import pandas as pd
import streamlit as st
from sqlalchemy import create_engine, text


# 数据库配置
database_username = ''
database_password = ''
database_ip = '127.0.0.1'
database_name = 'myemail'
# 创建数据库引擎
engine = create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}')

# 设置 API 密钥和基本 URL
client = openai.OpenAI(
    api_key="sk-",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

#
sentence_0 = """
        你是一个SQL查询生成助手。数据库的表名是emails2，列如下：
        - `Message-IDd` (VARCHAR(255)): 邮件唯一标识符，例如 '<18782981.1075855378110.JavaMail.evans@thyme'。
        - `Fromd` (VARCHAR(255)): 发件人邮件地址，例如 'phillip.allen@enron.co'。
        - `Tod` (VARCHAR(255)): 收件人邮件地址，例如 'tim.belden@enron.co'。
        - `Subjectd` (VARCHAR(255)): 邮件主题，可以为空，例如 'Re: test'。
        - `contentd` (LONGTEXT): 邮件内容，可能包含文本、换行符等。
        - `Timed` (DATETIME): 邮件日期和时间，格式例如 "2001-05-14 16:39:00"。
        请确保生成的SQL语句以```sql开头并以```结尾。
        """


def chat_with_model(instruct, freestyle):
    # 构造请求内容
    user_input = f"{instruct}{freestyle}"
    # 调用大模型进行对话
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="qwen-turbo"
    )
    # 获取模型回复
    model_reply = response.choices[0].message.content.strip()
    return model_reply


def extract_sql_query(complex_str):
    # 使用正则表达式提取SQL代码块
    match = re.search(r"```sql(.*?)```", complex_str, re.DOTALL)
    if match:
        sql_query = match.group(1).strip()
        return sql_query
    else:
        raise ValueError("模型返回的SQL语句格式不正确")


def execute_sql_get_df(sql_query):
    sql_query = sql_query.replace("\n", " ")
    sql_statements = sql_query.split(";")
    with engine.connect() as connection:
        for statement in sql_statements:
            if statement.strip():
                result = connection.execute(text(statement.strip()))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        return df


# Streamlit 应用
st.title("2024 邮件分析任务")
option = st.selectbox("选择功能:", ["<此处下拉菜单以选择功能>", "大模型邮件总结", "特定主题的邮件查询", "特定发件人的邮件聚合"])


if option == "此处下拉菜单选择功能":
    pass


elif option == "大模型邮件总结":
    st.markdown("# 大模型邮件总结")
    st.markdown("""
    * 利用Qwen等大语言模型，对检索到的邮件内容进行智能总结与生成,可以使用ollama结合qwen 0.5b的模型在本地运行。
    * 根据用户输入的检索条件(如关键词、时间范围、发件人等)，返回符合条件的邮件总结结果。
    * 总结结果要突出重点，忠实原文，语言通顺自然
    """)
    user_input = st.text_input("请在此输入:")

    if st.button("执行检索"):
        try:
            st.markdown("## SQL 语句:")
            user_input_2 = f"\n按照自然语言语义给出MySql语句：{user_input}"
            rough_sql_query = chat_with_model(sentence_0, user_input_2)
            sql_query = extract_sql_query(rough_sql_query)
            st.code(sql_query, language='sql')

            st.markdown("## 检索结果:")
            df_1 = execute_sql_get_df(sql_query)  # 返回的DataFrame有无限操作可能性
            st.dataframe(df_1)

            st.markdown("## 内容总结:")
            if 'contentd' in df_1.columns:
                all_content = "\n".join(df_1['contentd'].tolist())[:2000]  # 调整参数可以提高总结质量，但降低速度
                user_say = f"请概括性地总结这几封邮件内容: "
                model_say = chat_with_model(user_say, all_content)
                st.markdown(model_say)
            else:
                st.markdown("这个不需要总结。")

        except Exception as e:
            st.markdown(f"报错:\n{e}")


elif option == "特定主题的邮件查询":
    st.markdown("# 特定主题的邮件查询")
    st.markdown("* 针对特定主题的邮件查询，如\"最近一个月xxx@xxx.com的邮箱有没有谈到xxx内容\"。")
    user_input = st.text_input("请在此输入:")

    if st.button("执行查询"):
        try:
            st.markdown("## SQL 语句:")
            user_input_2 = f"\n按照自然语言的需求给出MySql语句：{user_input}"
            rough_sql_query = chat_with_model(sentence_0, user_input_2)
            sql_query = extract_sql_query(rough_sql_query)
            st.code(sql_query, language='sql')

            st.markdown("## 查询结果:")
            df_2 = execute_sql_get_df(sql_query)  # 返回的DataFrame有无限操作可能性
            st.dataframe(df_2)

            st.markdown("## 问题回答:")
            all_content = "\n".join(df_2['contentd'].tolist())[:2000]  # 调整参数可以提高总结质量，但降低速度
            user_say = f"回答问题“{user_input}”，参考下述文本：\n"
            model_say = chat_with_model(user_say, all_content)
            st.markdown(model_say)

        except Exception as e:
            st.markdown(f"报错:\n{e}")


elif option == "特定发件人的邮件聚合":
    st.markdown("# 特定发件人的邮件聚合")
    st.markdown("* 针对特定发件人的邮件聚合，如\"罗列出最近一个月xxx@xxx.com邮箱与其他人讨论的主要事项的前因后果\"，将完整的邮件沟通链条提取并进行总结,其中涉及内容的前因后果。")


# 侧边栏
st.sidebar.markdown("""
    <style>
    @keyframes example {
        0%   {color: red;}
        25%  {color: orange;}
        50%  {color: blue;}
        100% {color: green;}
    }
    .animated-header {
        font-size: 2em;
        font-weight: bold;
        animation-name: example;
        animation-duration: 10s;
        animation-iteration-count: infinite;
    }
    </style>
    <div class="animated-header">项目说明</div>
    <p>作者：</p>
    """, unsafe_allow_html=True)
st.sidebar.markdown("![1](https://raw.githubusercontent.com/LukiRyan/TyporaImageBox/main/img/202407191058169.png)")
